# Building a Formula One Dashboard

> *API: https://ergast.com/mrd/*

This is a workbook creating a `FormulaOne` class which is meant to streamline the API call for Formula One data.

The class is designed to scrape the API and save `.csv` files with the given schema below.

This workbook will be converted to a .py file in order to integrate with a dashboard.

## Tables

---

### Races
- Season
- Round
- URL
- RaceName
- CircuitID
- Date
- Time
- RaceID *(Created: Format {{year}{round}})*

### Circuits
- CircuitID
- URL
- CircuitName
- Latitude
- Longitude
- Locality
- Country

### Results
- RaceID
- Position
- Points
- DriverID
- ConstructorID
- Grid
- Laps
- Status
- Time
- FastestLapTime
- FastestLapSpeed


### Drivers
- DriverID
- Code
- URL
- FirstName
- LastName
- DOB
- Nationality

### Constructors
- ConstructorID
- URL
- Name
- Nationality

In [24]:
import requests
import json
import pandas as pd
import os
import datetime

# `class FormulaOne`

In [25]:
class FormulaOne:
    def __init__(self, 
                 data_directory='data', 
                 races_df='races.csv', 
                 circuits_df='circuits.csv', 
                 results_df='results.csv', 
                 drivers_df='drivers.csv', 
                 constructors_df='constructors.csv'):
        self.directory = data_directory
        self.races_df = races_df
        self.circuits_df = circuits_df
        self.results_df = results_df
        self.drivers_df = drivers_df
        self.constructors_df = constructors_df
        
        if self.directory in os.listdir():
            CSV_FILES = [races_df, circuits_df, results_df, 
                         drivers_df, constructors_df]
            if all([c in os.listdir(self.directory) for c in CSV_FILES]):
                self.races = pd.read_csv(
                    os.path.join(self.directory, races_df), index_col=0
                    )
                self.circuits = pd.read_csv(
                    os.path.join(self.directory, circuits_df), index_col=0
                    )
                self.results = pd.read_csv(
                    os.path.join(self.directory, results_df), index_col=0
                    )
                self.drivers = pd.read_csv(
                    os.path.join(self.directory, drivers_df), index_col=0
                    )
                self.constructors = pd.read_csv(
                    os.path.join(self.directory, constructors_df), index_col=0
                    )
            else:
                self.races = None
                self.circuits = None
                self.results = None
                self.drivers = None
                self.constructors = None
        else:
            os.mkdir(self.directory)
            self.races = None
            self.circuits = None
            self.results = None
            self.drivers = None
            self.constructors = None
            
    def _all_data_loaded(self):
        return (
            (not self.races is None) & 
            (not self.circuits is None) &
            (not self.results is None) &
            (not self.drivers is None) &
            (not self.constructors is None)
        )
        
    def _get_race_id(self, data):
        """
        Creates a RaceID from a season and round.
        Keys for ``data`` should be:
            ['season', 'round', 'url', 'raceName', 
             'Circuit', 'date', 'time', 'Results']
        """

        year = data.get('season', 0)
        rnd = data.get('round', 0)
        rnd = rnd if len(rnd) == 2 else f'0{rnd}'
        return int(f"{year}{rnd}")
    
    def _get_race(self, data):
        """
        Returns a row for the RACES table from a Races dictionary.
        Keys for ``data`` should be:
            ['season', 'round', 'url', 'raceName', 
             'Circuit', 'date', 'time', 'Results']
        """

        # Format datetime string.
        d = data.get('date', '1900-01-01')
        t = data.get('time', '01:01:01Z')
        dt_string = f'{d} {t}'

        # Return row.
        return dict(
            RaceID=self._get_race_id(data),
            Season=int(data.get('season', -1)),
            Round=int(data.get('round', -1)),
            RaceName=data.get('raceName'),
            CircuitID=data.get('Circuit').get('circuitId'),
            DateTime=pd.to_datetime(dt_string),
            URL=data.get('url'),
        )

    def _get_circuit(self, data):
        """
        Returns a row for the CIRCUIT table from a Races dictionary.
        Keys for ``data`` should be:
            ['season', 'round', 'url', 'raceName', 
             'Circuit', 'date', 'time', 'Results']
        """

        circuit = data.get('Circuit')
        return dict(
            CircuitID=circuit.get('circuitId'),
            CircuitName=circuit.get('circuitName'),
            Latitude=circuit.get('Location').get('lat'),
            Longitude=circuit.get('Location').get('long'),
            Locality=circuit.get('Location').get('locality'),
            Country=circuit.get('Location').get('country'),
            URL=circuit.get('url'),
        )
    
    def _get_results_drivers_constructors(self, data):
        """
        Returns all rows for a given RaceID for the RESULTS table 
            and all rows for a given RaceID for the DRIVERS table
            and all rows for a given RaceID for the CONSTRUCTORS table
            from a Races dictionary.
        Keys for ``data`` should be:
            ['season', 'round', 'url', 'raceName', 
             'Circuit', 'date', 'time', 'Results']
        """

        RESULTS = []
        DRIVERS = []
        CONSTRUCTORS = []
        race_id = self._get_race_id(data)

        results = data.get('Results')
        for r in results:
            driver = r.get('Driver')
            constructor = r.get('Constructor')

            results_row = dict(
                RaceID=race_id,
                Position=int(r.get('position', -1)),
                Points=int(r.get('points', -1)),
                DriverID=driver.get('driverId'),
                ConstructorID=constructor.get('constructorId'),
                Grid=int(r.get('grid', -1)),
                Laps=int(r.get('laps', -1)),
                Status=r.get('status'),
                Time=r.get('Time').get('time') if r.get('Time') else r.get('Time'),
                FastestLapTime=r.get('FastestLap').get('Time').get('time') \
                    if r.get('FastestLap') else None,
                FastestLapSpeed=r.get('FastestLap').get('AverageSpeed').get('speed') \
                    if r.get('FastestLap') else None
            )
            drivers_row = dict(
                DriverID=driver.get('driverId'),
                Code=driver.get('code'),
                FirstName=driver.get('givenName'),
                LastName=driver.get('familyName'),
                DOB=driver.get('dateOfBirth'),
                Nationality=driver.get('nationality'),
                URL=driver.get('url'),
            )
            constructors_row = dict(
                ConstructorID=constructor.get('constructorId'),
                Name=constructor.get('name'),
                Nationality=constructor.get('nationality'),
                URL=constructor.get('url'),
            )
            RESULTS.append(results_row)
            DRIVERS.append(drivers_row)
            CONSTRUCTORS.append(constructors_row)

        return RESULTS, DRIVERS, CONSTRUCTORS
    
    def _collect_data_from(self, year, round_num=1, verbose=True):
        """
        Collect all data from each event in a given year.

        Returns rows for tables:
            [RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS]
        """

        RACES = []
        CIRCUITS = []
        RESULTS = []
        DRIVERS = []
        CONSTRUCTORS = []

        while True:
            URL = f'http://ergast.com/api/f1/{year}/{round_num}/results.json'
            ROUND = requests.get(URL)
            if not ROUND.ok:
                if verbose:
                    print(f'\tComplete. Data from {len(RACES)} races compiled.')
                return RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS
            ROUND_DATA = json.loads(ROUND.content)
            data = ROUND_DATA['MRData']['RaceTable']['Races']

            if not data:
                if verbose:
                    print(f'\tComplete. Data from {len(RACES)} races compiled.')
                return RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS

            data = data[0]
            if verbose:
                print(f'Collecting data from: Year: {year} | Round: {round_num}')

            RACES.append(self._get_race(data))
            CIRCUITS.append(self._get_circuit(data))
            results, drivers, constructors = \
                self._get_results_drivers_constructors(data)
            RESULTS += results
            DRIVERS += drivers
            CONSTRUCTORS += constructors

            round_num += 1
            
    def _scrape_date_range(self, 
                           start_year, 
                           end_year=None,  
                           verbose=1):
        """
        Scrape data from a given ``start_year`` to a given ``end_year``.

        A directory path can be given as ``save_to`` in order to save the tables 
            in .csv format.
        """
        
        save_to_dir = self.directory
        
        if not end_year:
            end_year = datetime.datetime.now().year
        MICRO_VERBOSE = True if verbose == 2 else False

        ALL_RACES = []
        ALL_CIRCUITS = []
        ALL_RESULTS = []
        ALL_DRIVERS = []
        ALL_CONSTRUCTORS = []

        for year in range(start_year, end_year+1):
            if verbose:
                print(f'Scraping year {year}')
            races, circ, res, dr, const = self._collect_data_from(
                year, verbose=MICRO_VERBOSE)
            ALL_RACES += races
            ALL_CIRCUITS += circ
            ALL_RESULTS += res
            ALL_DRIVERS += dr
            ALL_CONSTRUCTORS += const

        races_df = pd.DataFrame(ALL_RACES)
        circuits_df = pd.DataFrame(ALL_CIRCUITS).drop_duplicates()
        results_df = pd.DataFrame(ALL_RESULTS)
        drivers_df = pd.DataFrame(ALL_DRIVERS).drop_duplicates()
        constructors_df = pd.DataFrame(ALL_CONSTRUCTORS).drop_duplicates()

        if save_to_dir:
            if save_to_dir not in os.listdir():
                os.mkdir(save_to_dir)

            races_df.to_csv(os.path.join(
                save_to_dir, self.races_df))
            circuits_df.to_csv(os.path.join(
                save_to_dir, self.circuits_df))
            results_df.to_csv(os.path.join(
                save_to_dir, self.results_df))
            drivers_df.to_csv(os.path.join(
                save_to_dir, self.drivers_df))
            constructors_df.to_csv(os.path.join(
                save_to_dir, self.constructors_df))

        return races_df, circuits_df, results_df, drivers_df, constructors_df

    def save_data(self):
        """Saves current internal data to ``self.directory``."""
        
        if not self._all_data_loaded():
            return
        
        # Races
        self.races = (
            self.races
            .sort_values('RaceID')
            .reset_index(drop=True)
            .drop_duplicates()
        )
        self.races.to_csv(os.path.join(
            self.directory, self.races_df))
        
        # Circuits
        self.circuits = (
            self.circuits
            .sort_values('CircuitID')
            .reset_index(drop=True)
            .drop_duplicates()
        )
        self.circuits.to_csv(os.path.join(
            self.directory, self.circuits_df))
        
        # Results
        self.results = (
            self.results
            .sort_values(['RaceID', 'Position'])
            .reset_index(drop=True)
            .drop_duplicates()
        )
        self.results.to_csv(os.path.join(
            self.directory, self.results_df))
        
        # Drivers
        self.drivers = (
            self.drivers
            .sort_values('DriverID')
            .reset_index(drop=True)
            .drop_duplicates()
        )
        self.drivers.to_csv(os.path.join(
            self.directory, self.drivers_df))
        
        # Constructors
        self.constructors = (
            self.constructors
            .sort_values('ConstructorID')
            .reset_index(drop=True)
            .drop_duplicates()
        )
        self.constructors.to_csv(os.path.join(
            self.directory, self.constructors_df))
        return f'`.csv` files saved to `{self.directory}`'
            
    def update(
        self, 
        last_results_url='http://ergast.com/api/f1/current/last/results.json'
            ):
        """Update and save data to the most recent season."""
        
        if not self._all_data_loaded():
            return 'No data loaded. Use `.scrape()` instead.'
        
        r = requests.get(last_results_url)
        if not r.ok:
            return f'Request could not be completed. | {r}'
        
        data = json.loads(r.content)['MRData']['RaceTable']['Races'][0]
        race_id = self._get_race_id(data)
        if race_id in set(self.races['RaceID']):
            return f'Data is up to date. | Most Recent RaceID: `{race_id}`'
        
        self.scrape(self.races['Season'].max())
        
    def scrape(self, start_year, end_year=None):
        """
        Scrape from start_year to end_year. Combine currently loaded data 
            and save to ``self.directory``.
        """
        
        (new_races, new_circuits, 
         new_results, new_drivers, 
         new_constructors) = self._scrape_date_range(start_year, end_year)
        
        if self._all_data_loaded():
            self.races = pd.concat(
                [self.races, new_races]).drop_duplicates()
            self.circuits = pd.concat(
                [self.circuits, new_circuits]).drop_duplicates()
            self.results = pd.concat(
                [self.results, new_results]).drop_duplicates()
            self.drivers = pd.concat(
                [self.drivers, new_drivers]).drop_duplicates()
            self.constructors = pd.concat(
                [self.constructors, new_constructors]).drop_duplicates()
        else:
            self.races = new_races
            self.circuits = new_circuits
            self.results = new_results
            self.drivers = new_drivers
            self.constructors = new_constructors
            
        self.save_data()
        return 'Complete!'
            
    def __repr__(self):
        return f'''Data
    Races
        {self.races.shape if not self.races is None else (0, 0)}
    Circuits
        {self.circuits.shape if not self.circuits is None else (0, 0)}
    Results
        {self.results.shape if not self.results is None else (0, 0)}
    Drivers
        {self.drivers.shape if not self.drivers is None else (0, 0)}
    Constructors
        {self.constructors.shape if not self.constructors is None else (0, 0)}
'''

In [26]:
f1 = FormulaOne()
f1

Data
    Races
        (144, 7)
    Circuits
        (27, 7)
    Results
        (2947, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

In [27]:
# f1.scrape(2014)

In [28]:
(f1.results
 .sort_values('FastestLapSpeed', ascending=False)
 .head(10)
 .merge(f1.races[['RaceID', 'CircuitID', 'RaceName', 'Season']], on='RaceID')
 .merge(f1.drivers[['DriverID', 'LastName']], on='DriverID')
 .merge(f1.circuits[['CircuitID', 'CircuitName']])
)[['FastestLapSpeed', 'RaceName', 'Season', 'LastName', 'CircuitName']]

,FastestLapSpeed,RaceName,Season,LastName,CircuitName
0,255.014,Italian Grand Prix,2019,Hamilton,Autodromo Nazionale di Monza
1,252.794,Italian Grand Prix,2018,Hamilton,Autodromo Nazionale di Monza
2,252.033,Italian Grand Prix,2020,Hamilton,Autodromo Nazionale di Monza
3,251.872,Italian Grand Prix,2019,Vettel,Autodromo Nazionale di Monza
4,252.770,Italian Grand Prix,2018,Vettel,Autodromo Nazionale di Monza
5,251.690,Italian Grand Prix,2019,Bottas,Autodromo Nazionale di Monza
6,251.544,Italian Grand Prix,2018,Bottas,Autodromo Nazionale di Monza
7,251.235,Italian Grand Prix,2019,Leclerc,Autodromo Nazionale di Monza
8,250.830,Italian Grand Prix,2019,Verstappen,Autodromo Nazionale di Monza
9,250.174,Italian Grand Prix,2017,Ricciardo,Autodromo Nazionale di Monza


In [29]:
f1.races.head()

,RaceID,Season,Round,RaceName,CircuitID,DateTime,URL
0,201401,2014,1,Australian Grand Prix,albert_park,2014-03-16 06:00:00+00:00,https://en.wikipedia.org/wiki/2014_Australian_...
1,201402,2014,2,Malaysian Grand Prix,sepang,2014-03-30 08:00:00+00:00,https://en.wikipedia.org/wiki/2014_Malaysian_G...
2,201403,2014,3,Bahrain Grand Prix,bahrain,2014-04-06 15:00:00+00:00,http://en.wikipedia.org/wiki/2014_Bahrain_Gran...
3,201404,2014,4,Chinese Grand Prix,shanghai,2014-04-20 07:00:00+00:00,http://en.wikipedia.org/wiki/2014_Chinese_Gran...
4,201405,2014,5,Spanish Grand Prix,catalunya,2014-05-11 12:00:00+00:00,http://en.wikipedia.org/wiki/2014_Spanish_Gran...


In [30]:
f1.circuits.head()

,CircuitID,CircuitName,Latitude,Longitude,Locality,Country,URL
0,BAK,Baku City Circuit,40.3725,49.85330,Baku,Azerbaijan,http://en.wikipedia.org/wiki/Baku_City_Circuit
1,albert_park,Albert Park Grand Prix Circuit,-37.8497,144.96800,Melbourne,Australia,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,americas,Circuit of the Americas,30.1328,-97.64110,Austin,USA,http://en.wikipedia.org/wiki/Circuit_of_the_Am...
3,bahrain,Bahrain International Circuit,26.0325,50.51060,Sakhir,Bahrain,http://en.wikipedia.org/wiki/Bahrain_Internati...
4,catalunya,Circuit de Barcelona-Catalunya,41.5700,2.26111,Montmeló,Spain,http://en.wikipedia.org/wiki/Circuit_de_Barcel...


In [31]:
f1.results.head()

,RaceID,Position,Points,DriverID,ConstructorID,Grid,Laps,Status,Time,FastestLapTime,FastestLapSpeed
0,201401,1,25,rosberg,mercedes,3,57,Finished,1:32:58.710,1:32.478,206.436
1,201401,2,18,kevin_magnussen,mclaren,4,57,Finished,+26.777,1:33.066,205.131
2,201401,3,15,button,mclaren,10,57,Finished,+30.027,1:32.917,205.460
3,201401,4,12,alonso,ferrari,5,57,Finished,+35.284,1:33.186,204.867
4,201401,5,10,bottas,williams,15,57,Finished,+47.639,1:32.616,206.128


In [32]:
f1.drivers.head()

,DriverID,Code,FirstName,LastName,DOB,Nationality,URL
0,aitken,AIT,Jack,Aitken,1995-09-23,British,http://en.wikipedia.org/wiki/Jack_Aitken
1,albon,ALB,Alexander,Albon,1996-03-23,Thai,http://en.wikipedia.org/wiki/Alexander_Albon
2,alonso,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
3,bottas,BOT,Valtteri,Bottas,1989-08-28,Finnish,http://en.wikipedia.org/wiki/Valtteri_Bottas
4,brendon_hartley,HAR,Brendon,Hartley,1989-11-10,New Zealander,http://en.wikipedia.org/wiki/Brendon_Hartley


In [33]:
f1.constructors.head()

,ConstructorID,Name,Nationality,URL
0,alfa,Alfa Romeo,Swiss,http://en.wikipedia.org/wiki/Alfa_Romeo_in_For...
1,alphatauri,AlphaTauri,Italian,http://en.wikipedia.org/wiki/Scuderia_AlphaTauri
2,alpine,Alpine F1 Team,French,http://en.wikipedia.org/wiki/Alpine_F1_Team
3,aston_martin,Aston Martin,British,http://en.wikipedia.org/wiki/Aston_Martin_in_F...
4,caterham,Caterham,Malaysian,http://en.wikipedia.org/wiki/Caterham_F1


In [34]:
f1.update()

'Data is up to date. | Most Recent RaceID: `202106`'

In [35]:
f1

Data
    Races
        (144, 7)
    Circuits
        (27, 7)
    Results
        (2947, 11)
    Drivers
        (51, 7)
    Constructors
        (19, 4)

# Work

In [ ]:
# r = requests.get('http://ergast.com/api/f1/current/last/results.json')
# r.ok

In [ ]:
# data = json.loads(r.content)['MRData']['RaceTable']['Races'][0]
# get_race_id(data)

In [ ]:
# get_race_id(data) in f1.races['RaceID']

In [ ]:
# def get_race_id(data):
#     """
#     Creates a RaceID from a season and round.
#     Keys for ``data`` should be:
#         ['season', 'round', 'url', 'raceName', 
#          'Circuit', 'date', 'time', 'Results']
#     """
    
#     year = data.get('season', 0)
#     rnd = data.get('round', 0)
#     rnd = rnd if len(rnd) == 2 else f'0{rnd}'
#     return int(f"{year}{rnd}")

In [ ]:
# # Races
# def get_race(data):
#     """
#     Returns a row for the RACES table from a Races dictionary.
#     Keys for ``data`` should be:
#         ['season', 'round', 'url', 'raceName', 
#          'Circuit', 'date', 'time', 'Results']
#     """
    
#     # Format datetime string.
#     d = data.get('date', '1900-01-01')
#     t = data.get('time', '01:01:01Z')
#     dt_string = f'{d} {t}'
    
#     # Return row.
#     return dict(
#         RaceID=get_race_id(data),
#         Season=int(data.get('season', -1)),
#         Round=int(data.get('round', -1)),
#         RaceName=data.get('raceName'),
#         CircuitID=data.get('Circuit').get('circuitId'),
#         DateTime=pd.to_datetime(dt_string),
#         URL=data.get('url'),
#     )
    

# # new_race = get_race(data)
# # new_race

In [ ]:
# # Circuit
# def get_circuit(data):
#     """
#     Returns a row for the CIRCUIT table from a Races dictionary.
#     Keys for ``data`` should be:
#         ['season', 'round', 'url', 'raceName', 
#          'Circuit', 'date', 'time', 'Results']
#     """
    
#     circuit = data.get('Circuit')
#     return dict(
#         CircuitID=circuit.get('circuitId'),
#         CircuitName=circuit.get('circuitName'),
#         Latitude=circuit.get('Location').get('lat'),
#         Longitude=circuit.get('Location').get('long'),
#         Locality=circuit.get('Location').get('locality'),
#         Country=circuit.get('Location').get('country'),
#         URL=circuit.get('url'),
#     )


# # new_circuit = get_circuit(data)
# # new_circuit

In [ ]:
# # Results / Drivers / Constructors
# def get_results_drivers_constructors(data):
#     """
#     Returns all rows for a given RaceID for the RESULTS table 
#         and all rows for a given RaceID for the DRIVERS table
#         and all rows for a given RaceID for the CONSTRUCTORS table
#         from a Races dictionary.
#     Keys for ``data`` should be:
#         ['season', 'round', 'url', 'raceName', 
#          'Circuit', 'date', 'time', 'Results']
#     """
    
#     RESULTS = []
#     DRIVERS = []
#     CONSTRUCTORS = []
#     race_id = get_race_id(data)
    
#     results = data.get('Results')
#     for r in results:
#         driver = r.get('Driver')
#         constructor = r.get('Constructor')
        
#         results_row = dict(
#             RaceID=race_id,
#             Position=int(r.get('position', -1)),
#             Points=int(r.get('points', -1)),
#             DriverID=driver.get('driverId'),
#             ConstructorID=constructor.get('constructorId'),
#             Grid=int(r.get('grid', -1)),
#             Laps=int(r.get('laps', -1)),
#             Status=r.get('status'),
#             Time=r.get('Time').get('time') if r.get('Time') else r.get('Time'),
#             FastestLapTime=r.get('FastestLap').get('Time').get('time') \
#                 if r.get('FastestLap') else None,
#             FastestLapSpeed=r.get('FastestLap').get('AverageSpeed').get('speed') \
#                 if r.get('FastestLap') else None
#         )
#         drivers_row = dict(
#             DriverID=driver.get('driverId'),
#             Code=driver.get('code'),
#             FirstName=driver.get('givenName'),
#             LastName=driver.get('familyName'),
#             DOB=driver.get('dateOfBirth'),
#             Nationality=driver.get('nationality'),
#             URL=driver.get('url'),
#         )
#         constructors_row = dict(
#             ConstructorsID=constructor.get('constructorId'),
#             Name=constructor.get('name'),
#             Nationality=constructor.get('nationality'),
#             URL=constructor.get('url'),
#         )
#         RESULTS.append(results_row)
#         DRIVERS.append(drivers_row)
#         CONSTRUCTORS.append(constructors_row)
    
#     return RESULTS, DRIVERS, CONSTRUCTORS


# # new_results, new_drivers, new_constructors = get_results_drivers_constructors(data)
# # new_results[0], new_drivers[0], new_constructors[0]

In [ ]:
# def collect_data_from(year, round_num=1, verbose=True):
#     """
#     Collect all data from each event in a given year.
    
#     Returns rows for tables:
#         [RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS]
#     """
    
#     RACES = []
#     CIRCUITS = []
#     RESULTS = []
#     DRIVERS = []
#     CONSTRUCTORS = []
    
#     while True:
#         URL = f'http://ergast.com/api/f1/{year}/{round_num}/results.json'
#         ROUND = requests.get(URL)
#         if not ROUND.ok:
#             if verbose:
#                 print(f'\tComplete. Data from {len(RACES)} races compiled.')
#             return RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS
#         ROUND_DATA = json.loads(ROUND.content)
#         data = ROUND_DATA['MRData']['RaceTable']['Races']
        
#         if not data:
#             if verbose:
#                 print(f'\tComplete. Data from {len(RACES)} races compiled.')
#             return RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS
            
#         data = data[0]
#         if verbose:
#             print(f'Collecting data from: Year: {year} | Round: {round_num}')

#         RACES.append(get_race(data))
#         CIRCUITS.append(get_circuit(data))
#         results, drivers, constructors = get_results_drivers_constructors(data)
#         RESULTS += results
#         DRIVERS += drivers
#         CONSTRUCTORS += constructors
        
#         round_num += 1

In [ ]:
# def scrape_date_range(start_year, end_year=2021, save_to_dir=None, verbose=1):
#     """
#     Scrape data from a given ``start_year`` to a given ``end_year``.
    
#     A directory path can be given as ``save_to`` in order to save the tables 
#         in .csv format.
#     """
    
#     MICRO_VERBOSE = True if verbose == 2 else False
    
#     ALL_RACES = []
#     ALL_CIRCUITS = []
#     ALL_RESULTS = []
#     ALL_DRIVERS = []
#     ALL_CONSTRUCTORS = []
    
#     for year in range(start_year, end_year+1):
#         if verbose:
#             print(f'Scraping year {year}')
#         races, circ, res, dr, const = collect_data_from(
#             year, verbose=MICRO_VERBOSE)
#         ALL_RACES += races
#         ALL_CIRCUITS += circ
#         ALL_RESULTS += res
#         ALL_DRIVERS += dr
#         ALL_CONSTRUCTORS += const
    
#     races_df = pd.DataFrame(ALL_RACES)
#     circuits_df = pd.DataFrame(ALL_CIRCUITS).drop_duplicates()
#     results_df = pd.DataFrame(ALL_RESULTS)
#     drivers_df = pd.DataFrame(ALL_DRIVERS).drop_duplicates()
#     constructors_df = pd.DataFrame(ALL_CONSTRUCTORS).drop_duplicates()
    
#     if save_to_dir:
#         if save_to_dir not in os.listdir():
#             os.mkdir(save_to_dir)
        
#         races_df.to_csv(os.path.join(save_to_dir, 'races.csv'))
#         circuits_df.to_csv(os.path.join(save_to_dir, 'circuits.csv'))
#         results_df.to_csv(os.path.join(save_to_dir, 'results.csv'))
#         drivers_df.to_csv(os.path.join(save_to_dir, 'drivers.csv'))
#         constructors_df.to_csv(os.path.join(save_to_dir, 'constructors.csv'))
    
#     return races_df, circuits_df, results_df, drivers_df, constructors_df

In [ ]:
# (RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS) = scrape_date_range(
#     2017, 2019)

# (NEW_RACES, NEW_CIRCUITS, NEW_RESULTS, 
#  NEW_DRIVERS, NEW_CONSTRUCTORS) = scrape_date_range(
#     2019, 2021)

In [ ]:
# pd.concat([RACES, NEW_RACES]).drop_duplicates()

In [ ]:
# RACES

In [ ]:
# (RACES, CIRCUITS, RESULTS, DRIVERS, CONSTRUCTORS) = collect_data_from(2019)

In [ ]:
# pd.DataFrame(RACES)

In [ ]:
# pd.DataFrame(CIRCUITS)

In [ ]:
# pd.DataFrame(RESULTS)

In [ ]:
# df = pd.DataFrame(DRIVERS)


In [ ]:
# df.drop_duplicates()

In [ ]:
# pd.DataFrame(CONSTRUCTORS).drop_duplicates()

In [ ]:
# DRIVERS.info()